# Caption Accuracy Metric with BLEU (Multi Edition)

In [1]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import pandas as pd
from data_loader import BLEUDataLoader
import time

In [2]:
ref_path="../captions/ref/gpt4v_llava_10k_test.json"
hypo_dir="../captions/hypo"

data_loader = BLEUDataLoader(ref_path=ref_path, ref_repeat=4, hypo_dir=hypo_dir, split=True)
# print(len(data_loader.ref), data_loader.ref[0])
# print(len(data_loader.hypo_dict["anti-0.01"]), data_loader.hypo_dict["anti-0.01"][0])


In [3]:
weights_list = [(1,0,0,0), (0,1,0,0), (0,0,1,0), (0,0,0,1), (0.25,0.25,0.25,0.25)]
data = []

for weights in weights_list:
    bleu_dict = {"weights": weights}
    for attr in data_loader.hypo_dict:
        bleu = corpus_bleu(data_loader.ref, data_loader.hypo_dict[attr],
                        weights=weights,
                        smoothing_function=None,
                        auto_reweigh=False)

        bleu_dict[attr] = bleu
    data.append(bleu_dict)

df_bleu = pd.DataFrame(data)
df_bleu

,weights,hypo-0.01
0,"(1, 0, 0, 0)",0.354741
1,"(0, 1, 0, 0)",0.120276
2,"(0, 0, 1, 0)",0.044844
3,"(0, 0, 0, 1)",0.020820
4,"(0.25, 0.25, 0.25, 0.25)",0.079445


In [4]:
csv_path = "../results/bleu_metric.csv"

df_bleu.to_csv(csv_path)

In [5]:
# for attr in data_loader.hypo_dict:
#     data = []
#     for i in range(len(data_loader.ref)):
#         bleu = sentence_bleu(data_loader.ref[i], data_loader.hypo_dict[attr][i],
#                     weights=(0.25,0.25,0.25,0.25),
#                     smoothing_function=None,
#                     auto_reweigh=False)
#         if len(data_loader.hypo_dict[attr][i]) <= 4:
#             print(data_loader.hypo_dict[attr][i])
#         record = {"bleu": bleu, "hypo": data_loader.hypo_dict[attr][i], "ref": data_loader.ref[i]}
#         data.append(record)
#     df = pd.DataFrame(data)
#     df.to_csv(f"../results/{attr}.csv")
#     print(f"{attr}: {df['bleu'].mean()}")
# df